In [1]:
import pandas as pd
from pathlib import Path

# Paths to input XLSB files by year
FILE_DIR = Path("./data")
FILE_PATHS = {
    2021: FILE_DIR / '2021 Detenidos_Nacional.xlsb',
    2022: FILE_DIR / '2022 Detenidos_Nacional.xlsb',
    2023: FILE_DIR / '2023 Detenidos_Nacional.xlsb',
    2024: FILE_DIR / '2024 Detenidos_Nacional.xlsb',
    2025: FILE_DIR / '2025 Detenidos_Nacional.xlsb'
}

# Population estimates by nationality for each year
POPULATION = {
    2021: {'Chilena': 17980000, 'Venezolana': 444423, 'Peruana': 246508, 'Haitiana': 180272, 'Colombiana': 173804, 'Boliviana': 132094},
    2022: {'Chilena': 17930000, 'Venezolana': 532715, 'Peruana': 250908, 'Haitiana': 184721, 'Colombiana': 189524, 'Boliviana': 148059},
    2023: {'Chilena': 17740000, 'Venezolana': 728586, 'Peruana': 260785, 'Haitiana': 188131, 'Colombiana': 209946, 'Boliviana': 180266},
    2024: {'Chilena': 16871782, 'Venezolana': 669408, 'Peruana': 233432, 'Haitiana': 80781,  'Colombiana': 197813, 'Boliviana': 168083},
    2025: {'Chilena': 16950000, 'Venezolana': 700000, 'Peruana': 240000, 'Haitiana': 85000,  'Colombiana': 210000, 'Boliviana': 170000}
}
TOTAL_POP = {
    2021: 19456334,
    2022: 19553036,
    2023: 19658835,
    2024: 18480432,
    2025: 18550000
}

# List of nationalities to analyze
NATIONALITIES = ['Chilena', 'Venezolana', 'Peruana', 'Haitiana', 'Colombiana', 'Boliviana']




In [10]:

def load_yearly_data(year: int, path: Path) -> pd.DataFrame:
    """
    Load the full detention data for a given year, preserving all columns.
    """
    print(f"Loading data for year {year} from {path}")
    xls = pd.ExcelFile(path, engine='pyxlsb')
    sheet = xls.sheet_names[0]
    # Read with header at row 6 to capture column names
    df = pd.read_excel(path, sheet_name=sheet, engine='pyxlsb', header=6)
    # Drop empty rows
    df = df.dropna(how='all').reset_index(drop=True)
    # Rename columns according to the file layout
    df.columns = ['',
        'Detenido', 'Región', 'Provincia', 'Comuna', 'Zonas', 'Prefectura', 'Destacamentos',
        'Año', 'Mes', 'Nacionalidad', 'Delitos o faltas', 'Total'
    ]
    # Filter by year
    df['Año'] = pd.to_numeric(df['Año'], errors='coerce')
    df = df[df['Año'] == int(year)]
    # Ensure Total is numeric
    df['Total'] = pd.to_numeric(df['Total'], errors='coerce').fillna(0).astype(int)
    return df




def compute_dpi(all_data: pd.DataFrame) -> pd.DataFrame:
    """
    Compute Disproportionality Parameter (DPI) for each offense-year-nationality.
    DPI = (Detention share) / (Population share)
    """
    records = []
    # Group by year and offense
    for (year, offense), df_off in all_data.groupby(['Año', 'Delitos o faltas']):
        total_off_dets = df_off['Total'].sum()
        if total_off_dets == 0:
            continue
        # Compute for each nationality
        for nat in NATIONALITIES:
            dets = df_off[df_off['Nacionalidad'].str.contains(nat, case=False, na=False)]['Total'].sum()
            det_share = dets / total_off_dets
            pop_share = POPULATION[year].get(nat, 0) / TOTAL_POP[year]
            dpi = det_share / pop_share if pop_share > 0 else None
            records.append({
                'Año': year,
                'Delitos o faltas': offense,
                'Nacionalidad': nat,
                'Detenciones': dets,
                'Poblacion': POPULATION[year].get(nat),
                'PartDetenciones': det_share,
                'PartPoblacion': pop_share,
                'DPI': round(dpi, 2) if dpi is not None else None
            })
    return pd.DataFrame(records)


In [11]:
data_frames = []
for year, path in FILE_PATHS.items():
    df_year = load_yearly_data(year, path)
    data_frames.append(df_year)
all_data = pd.concat(data_frames, ignore_index=True)

Loading data for year 2021 from data/2021 Detenidos_Nacional.xlsb
Loading data for year 2022 from data/2022 Detenidos_Nacional.xlsb
Loading data for year 2023 from data/2023 Detenidos_Nacional.xlsb
Loading data for year 2024 from data/2024 Detenidos_Nacional.xlsb
Loading data for year 2025 from data/2025 Detenidos_Nacional.xlsb


[        Year Nationality                                            Offense  \
 0       2021   Boliviana  Infringir normas higiénicas y de salubridad ar...   
 1       2021   Boliviana                                       Riña pública   
 2       2021     Chilena  Conducción bajo la influencia de alcohol con o...   
 3       2021     Chilena                      Conducción estado de ebriedad   
 4       2021     Chilena                                        Receptación   
 ...      ...         ...                                                ...   
 119798  2021     Chilena                    Aprehendidos por orden judicial   
 119799  2021     Chilena         Cuasidelito de Lesiones (Código agrupador)   
 119800  2021  Colombiana                  Conducción sin la licencia debida   
 119801  2021     Chilena     Amenazas simples contra personas y propiedades   
 119802  2021     Chilena         Cuasidelito de Lesiones (Código agrupador)   
 
         Total  
 0           1  
 1  

In [12]:
data_frames[1]


,,Detenido,Región,Provincia,Comuna,Zonas,Prefectura,Destacamentos,Año,Mes,Nacionalidad,Delitos o faltas,Total
1,NaN,DETENIDO,REGIÓN DE ARICA Y PARINACOTA,PROV. ARICA,ARICA,ZONA DE CARABINEROS DE ARICA Y PARINACOTA,PREFECTURA DE ARICA,1RA. COMISARÍA ARICA (U),2022.0,Enero,Argentina,Lesiones leves,1
2,NaN,DETENIDO,REGIÓN DE ARICA Y PARINACOTA,PROV. ARICA,ARICA,ZONA DE CARABINEROS DE ARICA Y PARINACOTA,PREFECTURA DE ARICA,1RA. COMISARÍA ARICA (U),2022.0,Enero,Boliviana,Conducción bajo la influencia de alcohol con o...,1
3,NaN,DETENIDO,REGIÓN DE ARICA Y PARINACOTA,PROV. ARICA,ARICA,ZONA DE CARABINEROS DE ARICA Y PARINACOTA,PREFECTURA DE ARICA,1RA. COMISARÍA ARICA (U),2022.0,Enero,Boliviana,Conducción estado de ebriedad,2
4,NaN,DETENIDO,REGIÓN DE ARICA Y PARINACOTA,PROV. ARICA,ARICA,ZONA DE CARABINEROS DE ARICA Y PARINACOTA,PREFECTURA DE ARICA,1RA. COMISARÍA ARICA (U),2022.0,Enero,Boliviana,Amenazas simples contra personas y propiedades,1
5,NaN,DETENIDO,REGIÓN DE ARICA Y PARINACOTA,PROV. ARICA,ARICA,ZONA DE CARABINEROS DE ARICA Y PARINACOTA,PREFECTURA DE ARICA,1RA. COMISARÍA ARICA (U),2022.0,Enero,Boliviana,Aprehendidos por orden judicial,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123979,NaN,DETENIDO,REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA,PROV. ULTIMA ESPERANZA,TORRES DEL PAINE,ZONA DE CARABINEROS MAGALLANES,PREFECTURA DE MAGALLANES,Retén Torres del Paine (F),2022.0,Marzo,Chilena,Lesiones graves,2
123980,NaN,DETENIDO,REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA,PROV. ULTIMA ESPERANZA,TORRES DEL PAINE,ZONA DE CARABINEROS MAGALLANES,PREFECTURA DE MAGALLANES,Retén Torres del Paine (F),2022.0,Abril,Otros Países,Uso Ilícito del Fuego,1
123981,NaN,DETENIDO,REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA,PROV. ULTIMA ESPERANZA,TORRES DEL PAINE,ZONA DE CARABINEROS MAGALLANES,PREFECTURA DE MAGALLANES,Retén Torres del Paine (F),2022.0,Junio,Chilena,Conducción estado de ebriedad,1
123982,NaN,DETENIDO,REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA,PROV. ULTIMA ESPERANZA,TORRES DEL PAINE,ZONA DE CARABINEROS MAGALLANES,PREFECTURA DE MAGALLANES,Retén Torres del Paine (F),2022.0,Agosto,Chilena,Aprehendidos por orden judicial,1


In [13]:
for item in data_frames:
    print(len(item))

119803
123983
128598
125696
33015


In [16]:
data_frames[0]['Prefectura'].unique()


array(['PREFECTURA DE ARICA', 'PREFECTURA DE IQUIQUE',
       'PREFECTURA DE ANTOFAGASTA', 'PREFECTURA DE EL LOA',
       'PREFECTURA DE ATACAMA', 'PREFECTURA DE LIMARÍ',
       'PREFECTURA DE COQUIMBO', 'PREFECTURA DE VALPARAÍSO',
       'PREFECTURA DE ACONCAGUA', 'PREFECTURA DE MARGA MARGA',
       'PREFECTURA DE SAN ANTONIO', 'PREFECTURA DE VIÑA DEL MAR',
       'PREFECTURA SANTIAGO NORTE', 'PREFECTURA SANTIAGO CORDILLERA',
       'PREFECTURA DEL MAIPO', 'PREFECTURA COSTA',
       'PREFECTURA SANTIAGO RINCONADA', 'PREFECTURA SANTIAGO OCCIDENTE',
       'PREFECTURA SANTIAGO SUR', 'PREFECTURA SANTIAGO CENTRAL',
       'PREFECTURA SANTIAGO ANDES', 'PREFECTURA SANTIAGO ORIENTE',
       'PREFECTURA DE CACHAPOAL', 'PREFECTURA DE COLCHAGUA',
       'PREFECTURA DE LINARES', 'PREFECTURA DE CURICÓ',
       'PREFECTURA DE TALCA', 'PREFECTURA DE ÑUBLE',
       'PREFECTURA DE ARAUCO', 'PREFECTURA DE BIOBíO',
       'PREFECTURA DE CONCEPCIÓN', 'PREFECTURA DE TALCAHUANO',
       'PREFECTURA DE CAU

In [17]:
data_frames[0]['Región'].unique()


array(['REGIÓN DE ARICA Y PARINACOTA', 'REGIÓN DE TARAPACÁ',
       'REGIÓN DE ANTOFAGASTA', 'REGIÓN DE ATACAMA', 'REGIÓN DE COQUIMBO',
       'REGIÓN DE VALPARAÍSO', 'REGIÓN METROPOLITANA DE SANTIAGO',
       "REGIÓN DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
       'REGIÓN DEL MAULE', 'REGIÓN DE ÑUBLE', 'REGIÓN DEL BIOBÍO',
       'REGIÓN DE LA ARAUCANÍA', 'REGIÓN DE LOS RÍOS',
       'REGIÓN DE LOS LAGOS',
       'REGIÓN DE AYSÉN DEL GENERAL CARLOS IBÁÑEZ DEL CAMPO',
       'REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA'], dtype=object)

In [18]:
data_frames[0]['Comuna'].unique()


array(['ARICA', 'CAMARONES', 'GENERAL LAGOS', 'PUTRE', 'ALTO HOSPICIO',
       'IQUIQUE', 'CAMIÑA', 'COLCHANE', 'HUARA', 'PICA', 'POZO ALMONTE',
       'ANTOFAGASTA', 'MEJILLONES', 'SIERRA GORDA', 'TALTAL', 'CALAMA',
       'OLLAGUE', 'SAN PEDRO DE ATACAMA', 'MARÍA ELENA', 'TOCOPILLA',
       'CHAÑARAL', 'DIEGO DE ALMAGRO', 'CALDERA', 'COPIAPÓ',
       'TIERRA AMARILLA', 'ALTO DEL CARMEN', 'FREIRINA', 'HUASCO',
       'VALLENAR', 'CANELA', 'ILLAPEL', 'LOS VILOS', 'SALAMANCA',
       'ANDACOLLO', 'COQUIMBO', 'LA HIGUERA', 'LA SERENA', 'PAIHUANO',
       'VICUÑA', 'COMBARBALÁ', 'MONTE PATRIA', 'OVALLE', 'PUNITAQUI',
       'RÍO HURTADO', 'ISLA DE PASCUA', 'CALLE LARGA', 'LOS ANDES',
       'RINCONADA', 'SAN ESTEBAN', 'LIMACHE', 'OLMUÉ', 'QUILPUÉ',
       'VILLA ALEMANA', 'CABILDO', 'LA LIGUA', 'PAPUDO', 'PETORCA',
       'ZAPALLAR', 'CALERA', 'HIJUELAS', 'LA CRUZ', 'NOGALES', 'QUILLOTA',
       'ALGARROBO', 'CARTAGENA', 'EL QUISCO', 'EL TABO', 'SAN ANTONIO',
       'SANTO DOMINGO', 'CATE

In [19]:
data_frames[0]['Zonas'].unique()

array(['ZONA DE CARABINEROS DE ARICA Y PARINACOTA',
       'ZONA DE CARABINEROS TARAPACÁ', 'ZONA DE CARABINEROS ANTOFAGASTA',
       'ZONA DE CARABINEROS ATACAMA', 'ZONA DE CARABINEROS COQUIMBO',
       'ZONA DE CARABINEROS VALPARAÍSO',
       'ZONA DE CARABINEROS SANTIAGO OESTE',
       'ZONA DE CARABINEROS SANTIAGO ESTE',
       "ZONA DE CARABINEROS LIB. BDO. O'HIGGINS",
       'ZONA DE CARABINEROS MAULE', 'ZONA DE CARABINEROS ÑUBLE',
       'ZONA DE CARABINEROS BIOBÍO', 'ZONA DE CARABINEROS ARAUCANÍA',
       'ZONA DE CARABINEROS DE LOS RÍOS', 'ZONA DE CARABINEROS LOS LAGOS',
       'ZONA DE CARABINEROS AYSÉN', 'ZONA DE CARABINEROS MAGALLANES'],
      dtype=object)